# Using jmzqc via Maven

In order to use jmzqc as a library, add the following statement to your maven pom.xml file. Please check https://ms-quality-hub.github.io/jmzqc/dependency-info.html for details on how to add the dependency for other build tools.

In [3]:
%%loadFromPOM
<!-- for mzQC functionality -->
<dependency>
    <groupId>org.lifs-tools</groupId>
    <artifactId>jmzqc</artifactId>
    <version>1.0.0-RC1</version>
</dependency>
<!-- for mzML access -->
<dependency>
    <groupId>io.github.msdk</groupId>
    <artifactId>msdk-io-mzml</artifactId>
    <version>0.0.27</version>
</dependency>
<!-- for table-based operations -->
<dependency>
    <groupId>tech.tablesaw</groupId>
    <artifactId>tablesaw-core</artifactId>
    <version>0.43.1</version>
</dependency>
<!-- for plotting -->
<dependency>
    <groupId>org.knowm.xchart</groupId>
    <artifactId>xchart</artifactId>
    <version>3.8.2</version>
</dependency>

UsageError: Cell magic `%%loadFromPOM` not found.


Add the following imports to your Java file to use the MzQC classes and the validation messages.

In [ ]:
import java.nio.*;
import java.nio.file.*;
import java.net.*;
import java.io.*;

In [ ]:
var outputDir = new File("proteomics-qc");
var file = new File(outputDir, "20181113_010_autoQC01.mzML");
var fileUrl = "https://massive.ucsd.edu/ProteoSAFe/DownloadResultFile?file=f.MSV000086542/ccms_peak/" + file.getName() + "&forceDownload=true";
outputDir.mkdirs();

try {
    InputStream in = new URL(fileUrl).openStream();
    Files.copy(in, file.toPath(), StandardCopyOption.REPLACE_EXISTING);
} catch (IOException ex) {
    System.err.println("Exception:" + ex.getLocalizedMessage());
}

In [ ]:
import io.github.msdk.MSDKException;
import io.github.msdk.MSDKRuntimeException;
import io.github.msdk.datamodel.ChromatogramType;
import io.github.msdk.io.mzml.MzMLFileImportMethod;
import io.github.msdk.io.mzml.data.MzMLRawDataFile;
import java.util.stream.Collectors;
List<MzMLRawDataFile> mzMLData = Collections.emptyList();
try {
    var mzMLFilePaths = Files.list(outputDir.toPath()).collect(Collectors.toList());
    mzMLData = mzMLFilePaths.stream().map(path -> {
        try {
            return new MzMLFileImportMethod(path).execute();
        } catch (MSDKException ex) {
            throw new MSDKRuntimeException(ex);
        }
    }
    ).collect(Collectors.toList());
} catch (IOException ex) {
    System.err.println("Exception:" + ex.getLocalizedMessage());
}

In [ ]:
import org.lifstools.jmzqc.*;
import com.networknt.schema.*;
import java.util.AbstractMap.SimpleEntry;
import com.google.common.collect.Range;
var mzMLFormatParameter = new CvParameter("MS:1000584", null, "mzML format", null);
Map<InputFile, List<QualityMetric>> mzMLFileStats = mzMLData.stream().map((t) -> {
    System.out.println("Processing file: " + t.getName());
    System.out.println("MS1 mz range...");
    var ms1MzRange = t.getScans().stream().filter(
            scan -> scan.getMsLevel() == 1
    ).map(
            scan -> scan.getMzRange()
    ).reduce(
            (l, r) -> l.span(r)
    ).orElse(Range.singleton(Double.NaN));

    var ms1MzRangeMetric = new QualityMetric("MS:4000069", null, "m/z acquisition range", Arrays.asList(ms1MzRange.lowerEndpoint(), ms1MzRange.upperEndpoint()), null);
    System.out.println("TIC and RT values...");
    var ticValuesAndRts = t.getChromatograms().stream().filter(
            chrom -> chrom.getChromatogramType() == ChromatogramType.TIC
    ).findFirst().map(
            chrom -> {
                return new SimpleEntry<>(chrom.getRetentionTimes(), chrom.getIntensityValues());
            }
    ).orElse(new SimpleEntry<>(new float[0], new float[0]));
    var totalIonChromatogram = new QualityMetric(
            "MS:1000235",
            null,
            "total ion current chromatogram",
            ticValuesAndRts.getValue(), new Unit(new CvParameter("UO:0000010", null, "second", ticValuesAndRts.getKey()), null));
    var numberOfChromatogramsMetric = new QualityMetric("MS:4000071", null, "number of chromatograms", t.getChromatograms().stream().count(), null);
    System.out.println("RT range...");
    var rtRange = t.getScans().stream().map(
            scan -> Range.singleton(scan.getRetentionTime())
    ).reduce(
            (lrt, rrt) -> lrt.span(rrt)
    ).get();
    var rtRangeMetric = new QualityMetric("MS:4000070", null, "retention time acquisition range", Arrays.asList(rtRange.lowerEndpoint(), rtRange.upperEndpoint()), new Unit(new CvParameter("UO:0000010", null, "second", null), null));
    return new SimpleEntry<InputFile, List<QualityMetric>>(
            new InputFile(mzMLFormatParameter, Collections.emptyList(), t.getOriginalFile().get().toURI(), t.getName()),
            Arrays.asList(
                    numberOfChromatogramsMetric,
                    ms1MzRangeMetric,
                    rtRangeMetric,
                    totalIonChromatogram
            )
    );
}).collect(Collectors.toMap(Map.Entry::getKey, Map.Entry::getValue));

In [ ]:
import com.fasterxml.jackson.annotation.JsonInclude.Include;
import com.fasterxml.jackson.core.JsonFactoryBuilder;
import com.fasterxml.jackson.core.JsonParser;
import com.fasterxml.jackson.core.JsonProcessingException;
import com.fasterxml.jackson.core.json.JsonReadFeature;
import com.fasterxml.jackson.databind.DeserializationContext;
import com.fasterxml.jackson.databind.JsonDeserializer;
import com.fasterxml.jackson.databind.ObjectMapper;
import com.fasterxml.jackson.databind.ObjectWriter;
import com.fasterxml.jackson.databind.SerializationFeature;
import com.fasterxml.jackson.databind.module.SimpleModule;
/* helper function to create formatted / pretty printed JSON */
public static ObjectWriter prepareJsonWriter() {
    JsonFactoryBuilder jfb = new JsonFactoryBuilder().
            enable(JsonReadFeature.ALLOW_TRAILING_COMMA);
    ObjectMapper mapper = new ObjectMapper(jfb.build());
    mapper.findAndRegisterModules();
    mapper.configure(SerializationFeature.WRITE_DATES_AS_TIMESTAMPS, false);
    mapper.configure(SerializationFeature.INDENT_OUTPUT, true);
    mapper.configure(JsonParser.Feature.ALLOW_COMMENTS, true);
    mapper.setSerializationInclusion(Include.NON_EMPTY);

    SimpleModule module = new SimpleModule();
    module.addDeserializer(OffsetDateTime.class, new JsonDeserializer<OffsetDateTime>() {
        @Override
        public OffsetDateTime deserialize(JsonParser jsonParser, DeserializationContext deserializationContext) throws IOException, JsonProcessingException {
            String value = jsonParser.getText();
            return Converter.parseDateTimeString(value);
        }
    });
    mapper.registerModule(module);
    return mapper.writerFor(Coordinate.class);
}

In [ ]:
import java.time.OffsetDateTime;
MzQC mzQC = null;
try {
    var analysisSoftware = new AnalysisSoftware("MS:1000799", null, "custom unreleased software tool", "jmzqc", new URI("https://github.com/MS-Quality-hub/jmzqc"), "1.0.0-RC1");
    List<BaseQuality> bqs = mzMLFileStats.entrySet().stream().map((t) -> {
        Metadata metadata = new Metadata(Arrays.asList(analysisSoftware), Collections.emptyList(), Arrays.asList(t.getKey()), null);
        return new BaseQuality(metadata, t.getValue());
    }
    ).collect(Collectors.toList());
    mzQC = new MzQC(
            "nils.hoffmann@cebitec.uni-bielefeld.de",
            "Nils Hoffmann",
            Arrays.asList(
                    new ControlledVocabulary(
                            "Proteomics Standards Initiative Mass Spectrometry Ontology",
                            new URI("https://github.com/HUPO-PSI/psi-ms-CV/releases/download/v4.1.103/psi-ms.obo"),
                            "4.1.103"
                    )
            ),
            OffsetDateTime.now(),
            "MzQC for basic QC information on MetaboLights dataset MTBLS1375",
            bqs,
            Collections.emptyList(),
            "1.0.0");
    Set<ValidationMessage> messages = Converter.validate(mzQC);
    System.out.println("Validation messages: " + messages);
    if(!messages.isEmpty()) {
      System.err.println("Validation failed with "+messages.size()+" messages!");
    } else {
      ObjectWriter writer = prepareJsonWriter();
      writer.writeValue(new File(file + ".mzQC"), new Coordinate(mzQC));
      System.out.println(writer.writeValueAsString(new Coordinate(mzQC)));
    }
} catch (URISyntaxException | IOException ex) {
    System.err.println("Exception: "+ex.getLocalizedMessage());
}

In [ ]:
import org.knowm.xchart.*;

if(mzQC!=null) {
  System.out.println("MzQC is not null!");
}
